In [1]:
import pandas as pd
import numpy as np
import json
import codecs
import preprocessing_tools as pr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_json('train.json',encoding = 'UTF-8')
# дропаем нейтральные отзывы
train = train[train.sentiment != 'neutral']
# Заменяем позитивный маркер на 1, негативный на 0
train['sentiment'] = train['sentiment'].replace(['positive' ,'negative'],[1,0])
# задаем целевую переменную
target = train['sentiment']

In [14]:
test_data = pd.read_json('test.json',encoding = 'UTF-8')

# Очищаем текст и лемматизируем
## Очистка

In [4]:
train['text'] = train['text'].apply(pr.clean_text)
test_data['text'] = test_data['text'].apply(pr.clean_text)

D:\Study\Diploma\Data_sentiment_kaggle\Data_sentiment\preprocessing_tools.py:31: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|[\"]", '', text)


## Лемматизация

In [5]:
%reload_ext autoreload
train['lemmas'] = train['text'].apply(pr.lemmatization)
test_data['lemmas'] = test_data['text'].apply(pr.lemmatization)

In [6]:
train

,id,sentiment,text,lemmas
0,1945,0,досудебное расследование по факту покупки енпф...,"[досудебный, расследование, факт, покупка, енп..."
1,1957,0,медики рассказали о состоянии пострадавшего му...,"[медик, состояние, пострадавший, мужчина, сове..."
2,1969,0,прошел почти год как железнодорожным оператора...,"[железнодорожный, оператор, запретить, эксплуа..."
3,1973,0,по итогам месяцев года на территории республ...,"[итог, месяц, территория, республика, выпустит..."
4,1975,0,астана ноября kazakhstan today агентство рк ...,"[астан, kazakhstan, today, агентство, рк, госу..."
5,1980,0,об аресте бывшего министра национальной эконом...,"[арест, бывший, министр, национальный, экономи..."
6,1982,0,двое налетчиков совершили нападение на охранни...,"[двое, налётчик, совершить, нападение, охранни..."
7,1983,0,самрукэнерго в i квартале почти вдвое снизила ...,"[самрукэнерго, квартал, вдвое, снизить, чистый..."
8,1985,0,в году почти половина коррупционных преступле...,"[половина, коррупционный, преступление, систем..."
9,1988,0,google чт янв один из фигурантов дела о кор...,"[google, чт, янв, фигурант, коррупция, ао, бай..."


# Векторизация

In [7]:
train['lemmas'] = train['lemmas'].apply(str)
test_data['lemmas'] = test_data['text'].apply(str)

In [8]:
tfidf_vectorizer = TfidfVectorizer(encoding='cp1251')

In [9]:
tfidf_vectorizer.fit(train['text'])
X = tfidf_vectorizer.transform(train['text'])
X_test = tfidf_vectorizer.transform(test_data['text'])

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)



C:\MyPrograms\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [18]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.667296786389414
Accuracy for C=0.05: 0.720226843100189
Accuracy for C=0.25: 0.831758034026465
Accuracy for C=0.5: 0.8695652173913043
Accuracy for C=1: 0.8846880907372401


In [26]:
print(np.shape(X_test))
print(np.shape(target))
print(np.shape(y_val))
print(np.shape(X_val))

(2056, 106235)
(4229,)
(1058,)
(1058, 106235)


In [20]:
final_tfidf = LogisticRegression(C=1)
final_tfidf.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_tfidf.predict(X_test)))

ValueError: Found input variables with inconsistent numbers of samples: [4229, 2056]